In [2]:
import open3d as o3d
import math
import numpy as np
import itertools
from scipy.spatial import Delaunay
from functools import reduce

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


From txt to ply and visualize pointcloud

In [3]:
pcd = o3d.io.read_point_cloud("daylight_preprocessed_images/lettuce1_2024-07-15_09.jpg", format='xyz')
o3d.io.write_point_cloud("output.ply", pcd)
o3d.visualization.draw_geometries([pcd]) 

[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


Visualize also cartesian axis (x red, y green, z blue)

In [13]:
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
o3d.visualization.draw_geometries([pcd, axes])

Ground segmentation

In [14]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.02, #find the plane with the largest support
                                         ransac_n=3,
                                         num_iterations=10000)
[a, b, c, d] = plane_model
plane_pcd = pcd.select_by_index(inliers) #select points by index
plane_pcd.paint_uniform_color([1.0, 0, 0])
lettuce_pcd = pcd.select_by_index(inliers, invert=True) #all except ground
lettuce_pcd.paint_uniform_color([0, 0, 1.0]) #paint with different colors lettuce and grounds
o3d.visualization.draw_geometries([plane_pcd, axes, lettuce_pcd])

Centering the axes with the pointcloud

In [15]:
plane_pcd = plane_pcd.translate((0,0,d/c)) #where ax+by+cz+d=0 is the plane
lettuce_pcd = lettuce_pcd.translate((0,0,d/c))
cos_theta = c / math.sqrt(a**2 + b**2 + c**2)
sin_theta = math.sqrt((a**2+b**2)/(a**2 + b**2 + c**2))
u_1 = b / math.sqrt(a**2 + b**2 )
u_2 = -a / math.sqrt(a**2 + b**2)
rotation_matrix = np.array([[cos_theta + u_1**2 * (1-cos_theta), u_1*u_2*(1-cos_theta), u_2*sin_theta],
                            [u_1*u_2*(1-cos_theta), cos_theta + u_2**2*(1- cos_theta), -u_1*sin_theta],
                            [-u_2*sin_theta, u_1*sin_theta, cos_theta]])
plane_pcd.rotate(rotation_matrix)
lettuce_pcd.rotate(rotation_matrix)
o3d.visualization.draw_geometries([plane_pcd, lettuce_pcd, axes])

Isolate one plant 

In [18]:
bounds = [[-0.3, 0.3], [0, 0.55], [0, -2]]
bounding_box_points = list(itertools.product(*bounds))
bounding_box = o3d.geometry.AxisAlignedBoundingBox.create_from_points(
        o3d.utility.Vector3dVector(bounding_box_points))
lettuce_croped = lettuce_pcd.crop(bounding_box)
o3d.visualization.draw_geometries([lettuce_croped])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Handle non valido. 


Remove outliers

In [21]:
cl, ind = lettuce_croped.remove_statistical_outlier(nb_neighbors=60,
                                                    std_ratio=2.0)
lettuce_croped = lettuce_croped.select_by_index(ind)
o3d.visualization.draw_geometries([lettuce_croped])

Define a mesh to simmulate the surface

In [23]:
downpdc = lettuce_croped.voxel_down_sample(voxel_size=0.05)
xyz = np.asarray(downpdc.points)
xy_catalog = []
for point in xyz:
    xy_catalog.append([point[0], point[1]])
tri = Delaunay(np.array(xy_catalog))

In [24]:
surface = o3d.geometry.TriangleMesh()
surface.vertices = o3d.utility.Vector3dVector(xyz)
surface.triangles = o3d.utility.Vector3iVector(tri.simplices)
o3d.visualization.draw_geometries([surface], mesh_show_wireframe=True)

In [25]:
def get_triangles_vertices(triangles, vertices):
    triangles_vertices = []
    for triangle in triangles:
        new_triangles_vertices = [vertices[triangle[0]], vertices[triangle[1]], vertices[triangle[2]]]
        triangles_vertices.append(new_triangles_vertices)
    return np.array(triangles_vertices)

In [26]:
def volume_under_triangle(triangle):
    p1, p2, p3 = triangle
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    x3, y3, z3 = p3
    return abs((z1+z2+z3)*(x1*y2-x2*y1+x2*y3-x3*y2+x3*y1-x1*y3)/6)

Estimate the Volume

In [27]:
volume = reduce(lambda a, b:  a + volume_under_triangle(b), get_triangles_vertices(surface.triangles, surface.vertices), 0)
print(f"The volume is: {round(volume, 4)} m3")

The volume is: 0.0013 m3
